In [4]:
import pandas as pd 
import numpy as np
# load data
data = pd.read_csv('Data/instagram_data.csv')
data.head()
data.shape

# check for missing values
data.isnull().sum()
data['image_path'] = data['image_path'].str.replace('../Data/', 'Data/')
data.head()

,likes,no_of_comments,t,follower_count_at_t,image_path
0,154552,0,1594174009,40934474,Data/insta_data/0.jpg
1,97386,0,1593571666,40934474,Data/insta_data/2.jpg
2,145632,0,1593136341,40934474,Data/insta_data/4.jpg
3,76461,0,1592981047,40934474,Data/insta_data/6.jpg
4,174620,0,1592703461,40934474,Data/insta_data/8.jpg


In [5]:
data.describe()

,likes,no_of_comments,t,follower_count_at_t
count,3.785000e+03,3785.000000,3.785000e+03,3.785000e+03
mean,1.832536e+05,2531.382827,1.585137e+09,1.409485e+07
std,1.936969e+05,21064.081137,1.322603e+07,9.402605e+06
min,1.431000e+03,0.000000,1.489547e+09,1.870000e+02
25%,5.208700e+04,194.000000,1.582540e+09,7.296298e+06
50%,1.236640e+05,484.000000,1.588855e+09,1.104970e+07
75%,2.431440e+05,1337.000000,1.592738e+09,1.836292e+07
max,2.161369e+06,733973.000000,1.594541e+09,4.093447e+07


In [6]:
# clean data
# let convert time to more readable format
data['time'] = pd.to_datetime(data['t'], unit='s')
data['hour'] = data['time'].dt.hour
data['day'] = data['time'].dt.dayofweek
data.head()


,likes,no_of_comments,t,follower_count_at_t,image_path,time,hour,day
0,154552,0,1594174009,40934474,Data/insta_data/0.jpg,2020-07-08 02:06:49,2,2
1,97386,0,1593571666,40934474,Data/insta_data/2.jpg,2020-07-01 02:47:46,2,2
2,145632,0,1593136341,40934474,Data/insta_data/4.jpg,2020-06-26 01:52:21,1,4
3,76461,0,1592981047,40934474,Data/insta_data/6.jpg,2020-06-24 06:44:07,6,2
4,174620,0,1592703461,40934474,Data/insta_data/8.jpg,2020-06-21 01:37:41,1,6


In [7]:
# # lets create a new column for engagement rate
data['engagement_rate'] = data['likes']/ data['follower_count_at_t']
data[['likes', 'no_of_comments', 'follower_count_at_t', 'engagement_rate']].describe()

,likes,no_of_comments,follower_count_at_t,engagement_rate
count,3.785000e+03,3785.000000,3.785000e+03,3785.000000
mean,1.832536e+05,2531.382827,1.409485e+07,5.690440
std,1.936969e+05,21064.081137,9.402605e+06,43.461269
min,1.431000e+03,0.000000,1.870000e+02,0.000136
25%,5.208700e+04,194.000000,7.296298e+06,0.003964
50%,1.236640e+05,484.000000,1.104970e+07,0.010654
75%,2.431440e+05,1337.000000,1.836292e+07,0.024636
max,2.161369e+06,733973.000000,4.093447e+07,603.352941


In [8]:
# since variance for num of comments are wide lets take a look
zero_comments = data[data['no_of_comments'] == 0]
zero_comments.shape
data = data[data['no_of_comments'] != 0]
data[['likes', 'no_of_comments', 'follower_count_at_t', 'engagement_rate']].describe()

,likes,no_of_comments,follower_count_at_t,engagement_rate
count,3.723000e+03,3723.000000,3.723000e+03,3723.000000
mean,1.837614e+05,2573.538544,1.382029e+07,5.785105
std,1.945265e+05,21236.241363,9.072975e+06,43.815512
min,1.431000e+03,1.000000,1.870000e+02,0.000136
25%,5.193050e+04,202.000000,7.248339e+06,0.004039
50%,1.236460e+05,500.000000,1.104970e+07,0.010823
75%,2.439885e+05,1362.000000,1.836292e+07,0.025045
max,2.161369e+06,733973.000000,4.022055e+07,603.352941


In [9]:
# there is abnormally high engagement rate, lets take a look
Q1 = data['likes'].quantile(0.25)
Q3 = data['likes'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 2 * IQR
upper_bound = Q3 + 2 * IQR
data = data[(data['likes'] >= lower_bound) & (data['likes'] <= upper_bound)]
data = data.reset_index(drop=True)
data[['likes', 'no_of_comments', 'follower_count_at_t', 'engagement_rate']].describe()


,likes,no_of_comments,follower_count_at_t,engagement_rate
count,3591.000000,3591.000000,3.591000e+03,3591.000000
mean,159059.322751,2244.492063,1.371471e+07,5.995492
std,139797.964615,21330.412206,9.087898e+06,44.599765
min,1431.000000,1.000000,1.870000e+02,0.000136
25%,49738.000000,196.500000,7.173292e+06,0.003896
50%,116176.000000,465.000000,1.087501e+07,0.010235
75%,223631.500000,1231.000000,1.814516e+07,0.022824
max,627073.000000,733973.000000,4.022055e+07,603.352941


In [10]:
# now lets process images
# import tensorflow as tf
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.applications.resnet50 import preprocess_input
# from tensorflow.keras.preprocessing import image
# import numpy as np
# import pandas as pd
# import os


# model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# def preprocess(image_path):
#     img = image.load_img(image_path, target_size=(224, 224))
#     img_array = image.img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0)
#     img_array = preprocess_input(img_array)
#     return img_array

# def extract_features(image_path):
#     img_array = preprocess(image_path)
#     features = model.predict(img_array)
#     return features.flatten()

# image_features = []
# for img in data['image_path']:
#     features = extract_features(img)
#     image_features.append(features)

# print(f"Length of image_features: {len(image_features)}")  
# print(f"Length of data: {len(data)}") 
# print(f"Shape of one feature vector: {np.array(image_features)[0].shape}")  

# image_data = pd.DataFrame(image_features, index=data.index)
# print(f"Shape of image_data DataFrame: {image_data.shape}") 

# image_data.columns = [f'feature_{i}' for i in range(image_data.shape[1])]

# data = pd.concat([data, image_data], axis=1)

# data.head()

In [21]:
# data.to_csv('Data/cleaned.csv', index=False)

In [23]:
from sklearn.cluster import KMeans

data = pd.read_csv('Data/cleaned.csv')

data['likes_class'] = pd.qcut(data['likes'], q=3, labels=[0, 1, 2])
class_distribution = data['likes_class'].value_counts()

features = ['no_of_comments', 'follower_count_at_t', 'hour', 'day', 'engagement_rate']
all_features = features + [f'feature_{i}' for i in range(2048)]

clustering_features = data[all_features]
kmeans = KMeans(n_clusters=3, random_state=42)
data['cluster_class'] = kmeans.fit_predict(clustering_features)
cluster_distribution = data['cluster_class'].value_counts()

print(f"class_distribution: {class_distribution}, cluster_distribution: {cluster_distribution}")


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


class_distribution: likes_class
0    1197
1    1197
2    1197
Name: count, dtype: int64, cluster_distribution: cluster_class
1    2035
0    1092
2     464
Name: count, dtype: int64


In [24]:
# lets do a simple classification using Random Forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

X = data[all_features]
y = data['likes_class']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


clf = RandomForestClassifier(n_estimators=100, random_state=42)

clf.fit(X_train, y_train)
y_pred_class = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_class)
print(f"Accuracy: {accuracy}")

report = classification_report(y_test, y_pred_class)
print("Classification Report:\n", report)

conf_matrix = confusion_matrix(y_test, y_pred_class)
print("Confusion Matrix:\n", conf_matrix)

(2872, 2053) (719, 2053) (2872,) (719,)
Accuracy: 0.6411682892906815
Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.78      0.74       238
           1       0.53      0.44      0.48       250
           2       0.66      0.72      0.69       231

    accuracy                           0.64       719
   macro avg       0.63      0.65      0.64       719
weighted avg       0.63      0.64      0.63       719

Confusion Matrix:
 [[185  43  10]
 [ 65 109  76]
 [ 11  53 167]]


In [25]:
# lets do hyperparameter tuning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(100, 500),  
    'max_depth': [None, 10, 20, 30, 40],  
    'min_samples_split': randint(2, 20),  
    'min_samples_leaf': randint(1, 20), 
    'class_weight': [None, 'balanced', 'balanced_subsample'],  
}

rf_classifier = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_dist,
                                   n_iter=50, cv=5, n_jobs=-1, verbose=2, scoring='accuracy', random_state=42)

random_search.fit(X_train, y_train)

best_rf_params = random_search.best_params_
print("Best Hyperparameters from Randomized Search:\n", best_rf_params)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END class_weight=balanced_subsample, max_depth=30, min_samples_leaf=15, min_samples_split=12, n_estimators=171; total time=   6.9s
[CV] END class_weight=balanced_subsample, max_depth=30, min_samples_leaf=15, min_samples_split=12, n_estimators=171; total time=   6.9s
[CV] END class_weight=balanced_subsample, max_depth=30, min_samples_leaf=15, min_samples_split=12, n_estimators=171; total time=   7.0s
[CV] END class_weight=balanced_subsample, max_depth=30, min_samples_leaf=15, min_samples_split=12, n_estimators=171; total time=   7.1s
[CV] END class_weight=balanced_subsample, max_depth=30, min_samples_leaf=15, min_samples_split=12, n_estimators=171; total time=   7.3s
[CV] END class_weight=None, max_depth=40, min_samples_leaf=7, min_samples_split=12, n_estimators=187; total time=   8.6s
[CV] END class_weight=None, max_depth=40, min_samples_leaf=7, min_samples_split=12, n_estimators=187; total time=   8.8s
[CV] END class_w

In [27]:
# Train the model with the best parameters from grid search
best_rf_classifier = RandomForestClassifier(**best_rf_params, random_state=42)
best_rf_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the model with tuned hyperparameters
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
print(f"Accuracy with Best Parameters: {accuracy_best_rf}")


report_best_rf = classification_report(y_test, y_pred_best_rf)
conf_matrix_best_rf = confusion_matrix(y_test, y_pred_best_rf)
print("Classification Report with Best Parameters:\n", report_best_rf)
print("Confusion Matrix with Best Parameters:\n", conf_matrix_best_rf)


Accuracy with Best Parameters: 0.694019471488178
Classification Report with Best Parameters:
               precision    recall  f1-score   support

           0       0.75      0.84      0.79       238
           1       0.61      0.46      0.53       250
           2       0.70      0.80      0.75       231

    accuracy                           0.69       719
   macro avg       0.69      0.70      0.69       719
weighted avg       0.68      0.69      0.68       719

Confusion Matrix with Best Parameters:
 [[199  37   2]
 [ 57 115  78]
 [ 10  36 185]]


In [28]:
# lets try xgboost
# import xgboost as xgb
# from sklearn.model_selection import GridSearchCV

# xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=3, random_state=42)

# # parameters
# xgb_param_grid = {
#     'n_estimators': [100, 200],
#     'learning_rate': [0.01, 0.1],
#     'max_depth': [3, 6],
#     'subsample': [0.8],
#     'colsample_bytree': [0.8]
# }

# xgb_grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=xgb_param_grid,
#                                cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# xgb_grid_search.fit(X_train, y_train)

# best_xgb_params = xgb_grid_search.best_params_
# print("Best Hyperparameters for XGBoost:\n", best_xgb_params)


In [ ]:

# tuned_xgb_classifier = xgb.XGBClassifier(**best_xgb_params, random_state=42)
# tuned_xgb_classifier.fit(X_train, y_train)

# y_pred_tuned_xgb = tuned_xgb_classifier.predict(X_test)

# tuned_xgb_accuracy = accuracy_score(y_test, y_pred_tuned_xgb)
# print(f"Accuracy with Tuned XGBoost: {tuned_xgb_accuracy}")

# tuned_xgb_report = classification_report(y_test, y_pred_tuned_xgb)
# tuned_xgb_conf_matrix = confusion_matrix(y_test, y_pred_tuned_xgb)
# print("Classification Report with Tuned XGBoost:\n", tuned_xgb_report)
# print("Confusion Matrix with Tuned XGBoost:\n", tuned_xgb_conf_matrix)


In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

if y_train.dtype.name == 'category':
    y_train = y_train.cat.codes
if y_test.dtype.name == 'category':
    y_test = y_test.cat.codes

nn_model = Sequential()
nn_model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
nn_model.add(BatchNormalization())
nn_model.add(Dropout(0.3))

nn_model.add(Dense(256, activation='relu'))
nn_model.add(BatchNormalization())
nn_model.add(Dropout(0.3))

nn_model.add(Dense(128, activation='relu'))
nn_model.add(BatchNormalization())
nn_model.add(Dropout(0.3))

nn_model.add(Dense(3, activation='softmax'))  

nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

nn_history = nn_model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

nn_accuracy = nn_model.evaluate(X_test, y_test)
print(f"Accuracy with Neural Network: {nn_accuracy[1]}")

y_pred_probabilities = nn_model.predict(X_test)
y_pred_nn = np.argmax(y_pred_probabilities, axis=1)
nn_report = classification_report(y_test, y_pred_nn)
nn_conf_matrix = confusion_matrix(y_test, y_pred_nn)
print("Classification Report with Neural Network:\n", nn_report)
print("Confusion Matrix with Neural Network:\n", nn_conf_matrix)

Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3491 - loss: 1.5781 - val_accuracy: 0.3652 - val_loss: 1.1192
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3651 - loss: 1.3025 - val_accuracy: 0.3530 - val_loss: 1.2160
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3453 - loss: 1.2874 - val_accuracy: 0.3930 - val_loss: 1.0896
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3405 - loss: 1.2421 - val_accuracy: 0.3739 - val_loss: 1.0841
Epoch 5/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3442 - loss: 1.1862 - val_accuracy: 0.4000 - val_loss: 1.1007
Epoch 6/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3622 - loss: 1.1728 - val_accuracy: 0.3270 - val_loss: 2.4636
Epoch 7/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3481 - loss: 1.1759 - val_accuracy: 0.3687 - val_loss: 1.0875
Epoch 8/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3577 - loss: 1.1667 - val_accuracy: 0.3809 - val_loss: 1.0771
Epo

In [31]:
# lets do CLIP model
# import torch
# import clip
# from PIL import Image
# import pandas as pd
# import numpy as np
# import os


# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/32", device=device)

# def extract_clip_features(image_path):
#     image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
#     features = model.encode_image(image)
#     return features.cpu().detach().numpy().flatten()

# clip_features = []
# for img in data['image_path']:
#     features = extract_clip_features(img)
#     clip_features.append(features)

# print(f"Length of clip_features: {len(clip_features)}")
# print(f"Length of data: {len(data)}")
# print(f"Shape of one feature vector: {np.array(clip_features)[0].shape}")

# clip_data = pd.DataFrame(clip_features, index=data.index)
# print(f"Shape of clip_data DataFrame: {clip_data.shape}")

# clip_data.columns = [f'clip_feature_{i}' for i in range(512)]

# data = pd.concat([data, clip_data], axis=1)

# data.to_csv('Data/clip_cleaned_data.csv', index=False)

In [32]:
# lets redo the training using clip features
data = pd.read_csv('Data/clip_cleaned_data.csv')

data['likes_class'] = pd.qcut(data['likes'], q=3, labels=[0, 1, 2])
class_distribution = data['likes_class'].value_counts()

features = ['no_of_comments', 'follower_count_at_t', 'hour', 'day', 'engagement_rate']
all_features = features + [f'feature_{i}' for i in range(2048)]

X = data[all_features]
y = data['likes_class']  

X_train_clip, X_test_clip, y_train_clip, y_test_clip = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:

param_dist = {
    'n_estimators': randint(100, 500),  
    'max_depth': [None, 10, 20, 30, 40],  
    'min_samples_split': randint(2, 20),  
    'min_samples_leaf': randint(1, 20), 
    'class_weight': [None, 'balanced', 'balanced_subsample'],  
}

rf_classifier = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_dist,
                                   n_iter=50, cv=5, n_jobs=-1, verbose=2, scoring='accuracy', random_state=42)

random_search.fit(X_train_clip, y_train_clip)

best_rf_params = random_search.best_params_

best_rf_classifier = RandomForestClassifier(**best_rf_params, random_state=42)
best_rf_classifier.fit(X_train_clip, y_train_clip)

y_pred_best_rf = best_rf_classifier.predict(X_test_clip)

accuracy_best_rf = accuracy_score(y_test_clip, y_pred_best_rf)
print(f"Accuracy with Best Parameters: {accuracy_best_rf}")

report_best_rf = classification_report(y_test_clip, y_pred_best_rf)
conf_matrix_best_rf = confusion_matrix(y_test_clip, y_pred_best_rf)
print("Classification Report with Best Parameters:\n", report_best_rf)
print("Confusion Matrix with Best Parameters:\n", conf_matrix_best_rf)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END class_weight=balanced_subsample, max_depth=30, min_samples_leaf=15, min_samples_split=12, n_estimators=171; total time=   6.9s
[CV] END class_weight=balanced_subsample, max_depth=30, min_samples_leaf=15, min_samples_split=12, n_estimators=171; total time=   7.0s
[CV] END class_weight=balanced_subsample, max_depth=30, min_samples_leaf=15, min_samples_split=12, n_estimators=171; total time=   7.0s
[CV] END class_weight=balanced_subsample, max_depth=30, min_samples_leaf=15, min_samples_split=12, n_estimators=171; total time=   7.3s
[CV] END class_weight=balanced_subsample, max_depth=30, min_samples_leaf=15, min_samples_split=12, n_estimators=171; total time=   7.5s
[CV] END class_weight=None, max_depth=40, min_samples_leaf=7, min_samples_split=12, n_estimators=187; total time=   8.7s
[CV] END class_weight=None, max_depth=40, min_samples_leaf=7, min_samples_split=12, n_estimators=187; total time=   9.0s
[CV] END class_w

In [35]:
# lets do the training  
# from sklearn.model_selection import train_test_split

# features = ['no_of_comments', 'follower_count_at_t', 'hour', 'day', 'engagement_rate']
# image_features = [f'feature_{i}' for i in range(2048)]
# all_features = features + image_features
# target = 'likes'

# X = data[all_features]
# Y = data[target]

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# X_train.shape, X_test.shape, y_train.shape, y_test.shape


In [36]:
# from sklearn.ensemble import RandomForestRegressor

# rf = RandomForestRegressor(n_estimators=100, random_state=42)
# rf.fit(X_train, y_train)

# y_pred = rf.predict(X_test)

In [37]:
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# from sklearn.metrics import mean_absolute_error

# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)

# mse, r2, mae

In [38]:
# #lets do a training without image features
# X = data[features]
# Y = data[target]

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# rf = RandomForestRegressor(n_estimators=100, random_state=42)
# rf.fit(X_train, y_train)

# y_pred = rf.predict(X_test)

# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)

# mse, r2, mae

In [39]:
# # again train the model
# features = ['no_of_comments', 'follower_count_at_t', 'hour', 'day', 'engagement_rate']
# clip_features = [f'clip_feature_{i}' for i in range(512)]
# all_features = features + clip_features
# target = 'likes'

# X = data[all_features]
# Y = data[target]

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# rf = RandomForestRegressor(n_estimators=100, random_state=42)
# rf.fit(X_train, y_train)

# y_pred = rf.predict(X_test)


# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)

# mse, r2, mae